# Imports

In [1]:
import os
import sys
sys.path.append(os.path.dirname(os.getcwd()))
import yaml
from Bio.SeqUtils import MeltingTemp as mt

## Define the parameters


Once the configuration file has been set up we have to read its content:

In [2]:
config_file = "./configs/probe_design_SPOT_custom.yaml"
# config_file = "./SPOT/configs/probe_design_SPOT_custom.yaml"
with open(config_file, 'r') as yaml_file:
    config = yaml.safe_load(yaml_file)
dir_output = os.path.join(os.path.dirname(os.getcwd()), config["dir_output"]) # create the complete path for the output directory

## Oligo sequences generation

Now we can start to actually build the pipeline, we will start by generating all the possible oligos with length between the maximum and minimum value given belonging to the genes defined in the config file. The oligos will be saved in a nested dicionary with the following structure: 

[gene][oligo_id][oligo_feature].


In [3]:
from oligo_designer_toolsuite.database import CustomGenomicRegionGenerator, NcbiGenomicRegionGenerator, EnsemblGenomicRegionGenerator

# If the custom config file is selected
if config["source"] == "custom":
    region_generator_custom = CustomGenomicRegionGenerator(
        annotation_file=config["file_annotation"], 
        sequence_file=config["file_sequence"], 
        files_source=config["files_source"], 
        species=config["species"], 
        annotation_release=config["annotation_release"], 
        genome_assembly=config["genome_assembly"],
        dir_output=dir_output
    )
# If the Ncbi config file is selected
elif config["source"] == "ncbi":
    region_generator = NcbiGenomicRegionGenerator(
        taxon=config["taxon"],
        species=config["species"], 
        annotation_release=config["annotation_release"], 
        dir_output=dir_output
    )
# If the Ensembl config file is generated
elif config["source"] == "ensembl":
    region_generator = EnsemblGenomicRegionGenerator(
        species=config["species"], 
        annotation_release=config["annotation_release"], 
        dir_output=dir_output
    )

file_transcriptome = region_generator_custom.generate_transcript_reduced_representation(include_exon_junctions=True, exon_junction_size=2*config["oligo_length_max"])

In [4]:
from oligo_designer_toolsuite.database import OligoDatabase

# define the database class
oligo_database = OligoDatabase(
    file_fasta = file_transcriptome,
    oligo_length_min = config["oligo_length_min"],
    oligo_length_max = config["oligo_length_max"],
    min_oligos_per_region = config["min_oligos_per_gene"],
    files_source = region_generator_custom.files_source,
    species = region_generator_custom.species,
    annotation_release = region_generator_custom.annotation_release,
    genome_assembly = region_generator_custom.genome_assembly,
    n_jobs = 2,
    dir_output=dir_output
)

# read the genes file
if config["file_genes"] is None:
    warnings.warn(
        "No file containing the genes was provided, all the genes are ussed to generate the probes. This chioce can use a lot of resources."
    )
    genes = None
else:
    with open(config["file_genes"]) as handle:
        lines = handle.readlines()
        genes = [line.rstrip() for line in lines]
        
# generate the oligo sequences from gene transcripts
oligo_database.create_database(region_ids=genes) 

# alternative: load database from file
# oligo_database.load_oligo_database(file_database)

### Dictionary structure


In [5]:
gene = list(oligo_database.database.keys())[0]
oligo_id = list(oligo_database.database[gene].keys())[0]

sample_oligos_DB = {}
sample_oligos_DB[gene] = {}
sample_oligos_DB[gene][oligo_id] = oligo_database.database[gene][oligo_id]
print(sample_oligos_DB)

{'WASIR2': {'WASIR2_16:22915-22941(+)': {'sequence': Seq('TCACTTTGCGGCCACTATGAGGACTG'), 'chromosome': '16', 'start': [22915], 'end': [22941], 'strand': '+', 'length': 26, 'additional_information_fasta': ['transcript_id=NR_130735.1,exon_number=1']}}}


In [6]:
oligo_database.database.keys()

dict_keys(['WASIR2'])

In [7]:
def number_of_probes(oligo_database):
    d = dict()
    for i in oligo_database.database.keys():
        d[i] = len(oligo_database.database[i])
    print(d)

### Read and write



In [8]:
if config["write_intermediate_steps"]:
    file_database = oligo_database.write_database(filename="oligo_database_initial.txt")
number_of_probes(oligo_database)

{'WASIR2': 5060}


## Property filters

Once all the possible sequences are created, we apply a first filtering process based on the sequences properties (e.g. melting temperature or GC content). This is useful to reduce the amount of sequences we have to deal with in the next stages and discard all the sequences that are not suited for the experiment.

In [9]:
from oligo_designer_toolsuite.oligo_property_filter import (
    PropertyFilter,
    MaskedSequences,
    GCContent, 
    ProhibitedSequences
)
# initialize the filters clasees
masked_sequences = MaskedSequences()
gc_content = GCContent(GC_content_min=config["GC_content_min"], GC_content_max=config["GC_content_max"])
proh_seq = ProhibitedSequences(num_consecutive = config["number_consecutive"])

# create the list of filters
filters = [masked_sequences, proh_seq ,gc_content]

# initialize the property filter class
property_filter = PropertyFilter(filters=filters, write_regions_with_insufficient_oligos=config["write_removed_genes"])

# filter the database
oligo_database = property_filter.apply(oligo_database=oligo_database, n_jobs=config["n_jobs"])

# check for unsufficient number of probes
#oligo_database.remove_genes_with_insufficient_oligos(pipeline_step = "after applying property filters")

# write the intermediate result in a file
if config["write_intermediate_steps"]:
    file_database = oligo_database.write_database(filename="oligo_database_property_filter.txt")

In [10]:
number_of_probes(oligo_database)

{'WASIR2': 2032}


## Specificity filters


In [11]:
from oligo_designer_toolsuite.database import ReferenceDatabase
from oligo_designer_toolsuite.oligo_specificity_filter import (
    SpecificityFilter,
    ExactMatches,
    Blastn,
)
dir_specificity = os.path.join(dir_output, "specificity_temporary") # folder where the temporary files will be written


reference = ReferenceDatabase(
    file_fasta = file_transcriptome,
    files_source = region_generator_custom.files_source,
    species = region_generator_custom.species,
    annotation_release = region_generator_custom.annotation_release,
    genome_assembly = region_generator_custom.genome_assembly,
    dir_output=dir_output
    )

# intialize the filter classes
exact_mathces = ExactMatches(dir_specificity=dir_specificity)
blastn = Blastn(
    dir_specificity=dir_specificity, 
    word_size=config["word_size"],
    percent_identity=config["percent_identity"],
    coverage=config["coverage"],
    #strand=config["strand"],
    strand='plus',
)
filters = [exact_mathces,blastn]

# initialize the specificity filter class
specificity_filter = SpecificityFilter(filters=filters, write_regions_with_insufficient_oligos=config["write_removed_genes"])
# filte r the database
oligo_database = specificity_filter.apply(oligo_database=oligo_database, reference_database=reference, n_jobs=config["n_jobs"])
# write the intermediate result
if config["write_intermediate_steps"]:
    file_database = oligo_database.write_database(filename="oligo_database_specificity_filter.txt")

In [12]:
number_of_probes(oligo_database)

{'WASIR2': 104}


## Oligoset generation


In [14]:
from oligo_designer_toolsuite.oligo_efficiency import(
    SeqFISHOligoScoring,
    AverageSetScoring
)
from oligo_designer_toolsuite.oligo_selection import OligosetGenerator, padlock_heuristic_selection

# initialize the scoring classes
"""oligos_scoring = PadlockOligoScoring(
    Tm_min=config["Tm_min"],
    Tm_opt=config["Tm_opt"],
    Tm_max=config["Tm_max"],
    GC_content_min=config["GC_content_min"],
    GC_content_opt=config["GC_content_opt"],
    GC_content_max=config["GC_content_max"],
    Tm_weight=config["Tm_weight"],
    GC_weight=config["GC_weight"],
)
"""


oligos_scoring = SeqFISHOligoScoring(
    GC_content_min=config["GC_content_min"],
    GC_content_opt=config["GC_content_opt"],
    GC_content_max=config["GC_content_max"],
    GC_weight=config["GC_weight"],)
set_scoring = AverageSetScoring()

# initialize the oligoset generator class
oligoset_generator = OligosetGenerator(
    oligoset_size=config["oligoset_size"], 
    min_oligoset_size=config["min_oligoset_size"],
    oligos_scoring=oligos_scoring,
    set_scoring=set_scoring,
    heurustic_selection=padlock_heuristic_selection,
    write_regions_with_insufficient_oligos=config["write_removed_genes"]
)

# generate the oligoset
oligo_database = oligoset_generator.apply(oligo_database=oligo_database, n_sets=config["n_sets"], n_jobs=config["n_jobs"])
# write the intermediate result
if config["write_intermediate_steps"]:
    oligo_database.write_oligosets(dir_oligosets="oligosets")

TypeError: write_oligosets() got an unexpected keyword argument 'dir_oligosets'

In [17]:
oligo_database.oligosets

{}

## Total probe generator

In [15]:
# create readout probes
from oligo_designer_toolsuite.sequence_design.readout_probes_generator import ReadoutProbes
readout_generator = ReadoutProbes(length=15,  number_probes = 60, GC_min=45, GC_max=55,number_consecutive = 5, random_seed = 0, blast_filter = blastn, reference_DB = reference)
readout_probes = readout_generator.create_probes()

NameError: name 'res' is not defined

In [16]:
readout_probes[0]

NameError: name 'readout_probes' is not defined

In [17]:
from oligo_designer_toolsuite.sequence_design.barcoding_creation import BarcodingCreator
barcodes_creator = BarcodingCreator(20, list(oligo_database.database.keys()))
barcodes_for_genes =  barcodes_creator.create_barcodes()

In [18]:
barcodes_for_genes

{'WASIR2': [7, 9, 13, 9]}

In [20]:
# generate barcodes
from oligo_designer_toolsuite.sequence_design.seqFISH_probes_designer import SeqfishProbesCreator
probes_creator = SeqfishProbesCreator()
oligo_database.oligos_DB = probes_creator.create_probes(oligo_database.database, readout_probes, barcodes_for_genes)

NameError: name 'readout_probes' is not defined

In [17]:
oligo_database.oligos_DB['CYBA']['CYBA_10024']

{'sequence': Seq('CCATGCACTAGGCTTTCTCCCACGCTTGCC'),
 'transcript_id': ['XM_011522905.4'],
 'exon_id': ['XM_011522905.4_exon6'],
 'chromosome': '16',
 'start': [88643703],
 'end': [88643733],
 'strand': '-',
 'length': 30,
 'melting_temperature': 66.3320282696755,
 'GC_content': 60.0,
 'sequence_with_barcodes': Seq('CTATAGAGGGTCTACAGGTAATACGCACCGCCATGCACTAGGCTTTCTCCCACG...CCG')}

## !!! Add cross hybridization check here !!!

In [22]:
oligo_database.write_fasta_from_database(filename = 'fasta_from_our_db')

'/Users/anna/Desktop/Project/oligo-designer-toolsuite/output_SPOT_custom/oligo_database/fasta_from_our_db.fna'

In [23]:
ref_db = ReferenceDatabase(file_fasta ='/Users/anna/Desktop/Project/oligo-designer-toolsuite/output_SPOT_custom/oligo_database/fasta_from_our_db.fna')

/Users/anna/Desktop/Project/oligo-designer-toolsuite/oligo_designer_toolsuite/database/_reference_database.py:59: UserWarning: No files_source defined. Using default files_source custom!
  warnings.warn(
/Users/anna/Desktop/Project/oligo-designer-toolsuite/oligo_designer_toolsuite/database/_reference_database.py:65: UserWarning: No species defined. Using default species unknown!
  warnings.warn(f"No species defined. Using default species {species}!")
/Users/anna/Desktop/Project/oligo-designer-toolsuite/oligo_designer_toolsuite/database/_reference_database.py:69: UserWarning: No annotation release defined. Using default release unknown!
  warnings.warn(
/Users/anna/Desktop/Project/oligo-designer-toolsuite/oligo_designer_toolsuite/database/_reference_database.py:75: UserWarning: No genome assembly defined. Using default genome assembly unknown!
  warnings.warn(


In [24]:
exact_mathces = ExactMatches(dir_specificity=dir_specificity)
blastn = Blastn(
    dir_specificity=dir_specificity, 
    word_size=config["word_size"],
    percent_identity=config["percent_identity"],
    coverage=config["coverage"],
    #strand=config["strand"],
    strand='minus',
)
filters = [exact_mathces,blastn]

# initialize the specificity filter class
specificity_filter = SpecificityFilter(filters=filters, write_regions_with_insufficient_oligos=config["write_removed_genes"])
# filte r the database
oligo_database = specificity_filter.apply(oligo_database=oligo_database, reference_database=ref_db, n_jobs=config["n_jobs"])


In [26]:
number_of_probes(oligo_database)

{'WASIR2': 29}
